<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>LangChain</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from operator import itemgetter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import torch

import openai
from openai import OpenAI

import transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results


import langchain
from langchain.chains import create_sql_query_chain
from langchain.tools import DuckDuckGoSearchRun

import langchain_openai
from langchain_openai import ChatOpenAI

import langchain_anthropic
from langchain_anthropic import ChatAnthropic

import langchain_core
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough

import langchain_community
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

Compiler    : Clang 17.0.0 (clang-1700.0.13.3)
OS          : Darwin
Release     : 25.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 2563372cfdc6863bbd75970c20a0f7113c7e3c80

torch              : 2.7.0
langchain_openai   : 0.3.18
langchain_community: 0.3.24
numpy              : 2.2.5
matplotlib         : 3.10.3
watermark          : 2.5.0
transformers       : 4.52.3
langchain_core     : 0.3.62
openai             : 1.78.1
langchain          : 0.3.25
pandas             : 2.2.3
langchain_anthropic: 0.3.14



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')

# OpenAI

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key. Then we are ready to instantiate the client

In [4]:
client = OpenAI()

In [5]:
print("\n".join([model.id for model in client.models.list().data]))

gpt-4-0613
gpt-4
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
computer-use-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
computer-use-preview-2025-03-11
gpt-4o-search-preview-2025-03-11
gpt-4o-sea

In [6]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

In [7]:
print(response)

ChatCompletion(id='chatcmpl-D0Y5KbbEhAWhOEQG8DQxYrEsYh68Z', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's primary weakness is Kryptonite, a radioactive fragment from his home planet, Krypton. It comes in various forms, the most common being green Kryptonite, which weakens him and can be lethal with prolonged exposure. Other forms of Kryptonite, such as red or gold, have different effects, like altering his behavior or memory. Additionally, Superman can also be vulnerable to magic and red solar radiation, which can diminish his powers.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1769023394, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_a0e9480a2f', usage=CompletionUsage(completion_tokens=88, prompt_tokens=13, total_tokens=101, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0

In [8]:
print(response.choices[0].message.content)

Superman's primary weakness is Kryptonite, a radioactive fragment from his home planet, Krypton. It comes in various forms, the most common being green Kryptonite, which weakens him and can be lethal with prolonged exposure. Other forms of Kryptonite, such as red or gold, have different effects, like altering his behavior or memory. Additionally, Superman can also be vulnerable to magic and red solar radiation, which can diminish his powers.


# LangChain

We instantiate the LangChain interface for OpenAI

In [9]:
model = ChatOpenAI(model="gpt-4o")

In [10]:
messages = [
    SystemMessage(content="What was Superman's weakness?"),
]

output = model.invoke(messages)
print(output)

content="Superman's primary weakness is Kryptonite, a mineral from his home planet of Krypton. When exposed to Kryptonite, Superman loses his powers and can become severely weakened or even die if exposed for too long. Kryptonite comes in different forms and colors, each affecting Superman in various ways, but the most common and well-known is the green variety, which is lethal to him. Additionally, Superman is vulnerable to magic and can also be overpowered or harmed by beings of similar or greater strength." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 13, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-D0Y6O9GrFti5aFvzjV6WnDptHyw3n', 'service_tier': 'd

In [11]:
output.response_metadata["token_usage"]

{'completion_tokens': 99,
 'prompt_tokens': 13,
 'total_tokens': 112,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

In [12]:
parser = StrOutputParser()

In [13]:
result = model.invoke(messages)

In [14]:
parser.invoke(result)

"Superman's primary weakness is kryptonite, a radioactive mineral from his home planet, Krypton. Exposure to kryptonite weakens Superman, stripping him of his powers, and prolonged exposure can be deadly. There are different types of kryptonite, with green kryptonite being the most well-known and commonly depicted. Other forms, such as red or gold kryptonite, have various effects on Superman, often temporary and unpredictable. In addition to kryptonite, Superman is vulnerable to magic, can be affected by the loss of solar energy from a yellow sun, and faces moral and ethical dilemmas that challenge his ideals."

Let us create our first chain. Stages of the chain are conencted with the pipe '|' character

In [15]:
chain = model | parser

Now whenver we call __invoke()__ on the chain, it automatically runs all the steps

In [16]:
chain.invoke(messages)

"Superman's primary weakness is kryptonite, a radioactive mineral from his home planet of Krypton. Exposure to kryptonite drains Superman's strength and can even incapacitate or kill him if he's exposed for a prolonged period. Different variations of kryptonite can have different effects: for example, green kryptonite weakens him, red kryptonite causes bizarre and unpredictable behavior changes, and gold kryptonite can strip him of his powers permanently. Apart from kryptonite, Superman is also vulnerable to magic, and he can be overpowered by beings of equal or greater strength."

We can also create templates for our prompts, following conventions similar to the Jinja templating system

In [17]:
system_template = "Translate the following text into {language}:"

And we can combine multiple messages into a single template

In [18]:
prompt_template = ChatPromptTemplate.from_messages(
    [
     ("system", system_template), 
     ("user", "{text}")
    ]
)

To instantiate the prompt, we must provide the correct fields

In [19]:
result = prompt_template.invoke(
    {
        "language": "italian", 
        "text": "Be the change that you wish to see in the world."
    }
)

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following text into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Be the change that you wish to see in the world.', additional_kwargs={}, response_metadata={})])

The full interaction is:

In [21]:
result.to_messages()

[SystemMessage(content='Translate the following text into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Be the change that you wish to see in the world.', additional_kwargs={}, response_metadata={})]

In [22]:
chain = prompt_template | model | parser

In [23]:
chain.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

'Sei addestrato su dati fino a ottobre 2023.'

# Anthropic

In [24]:
model_a = ChatAnthropic(model="claude-haiku-4-5-20251001")

In [25]:
model_a.invoke("What is Superman's weakness?")

AIMessage(content="Superman's primary weakness is **kryptonite**, radioactive fragments from his home planet Krypton. Exposure to it drains his powers and can be fatal.\n\nHe also has other notable weaknesses:\n\n- **Magic** - He has no natural resistance to magical attacks\n- **Red sun radiation** - Rays from a red sun (like Krypton's) nullify his powers\n- **Psychological vulnerabilities** - His concern for loved ones can be exploited\n- **Limited power in certain conditions** - His abilities depend on yellow sun radiation\n\nKryptonite is by far his most famous weakness and is frequently used against him in comics, films, and TV shows.", additional_kwargs={}, response_metadata={'id': 'msg_01Sx1W7VWhfbrTmcv3TTGazo', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 13, 'output_tokens': 151, 'server_tool_use': None, 'service_tier': 'standard', 'cache_creation

In [26]:
chain_a = prompt_template | model_a | parser

In [27]:
model_a

ChatAnthropic(model='claude-haiku-4-5-20251001', anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), model_kwargs={})

In [28]:
chain_a.invoke(
    {
        "language": "italian", 
        "text": "Be the change that you wish to see in the world."
    }
)

'# Sii il cambiamento che desideri vedere nel mondo.\n\nThis is a famous quote often attributed to Mahatma Gandhi, though its exact origin is debated. It emphasizes personal responsibility and the idea that transformation begins with oneself.'

# Message History

In [29]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [30]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [31]:
config = {"configurable": {"session_id": "abc2"}}

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bruno")],
    config=config,
)

response.content

'Hello Bruno! How can I assist you today?'

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You mentioned that your name is Bruno. How can I help you today, Bruno?'

In [34]:
store

{'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Bruno", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Bruno! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-D0YCebr1pnQOOjuJIviSPnWayRkxv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d079d1e2-8583-47af-85c4-06a94cf90194-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content="What's my nam

In [35]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I don't have access to personal data about individuals unless it's shared with me during the course of our conversation. If you tell me your name, I'll be happy to address you by it!"

In [36]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"You said your name is Bruno. Is there anything else you'd like to discuss or any way I can assist you?"

In [37]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [39]:
chain = prompt | model | parser

In [40]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response

'Hello, Bob! How can I assist you today?'

In [41]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [42]:
config = {"configurable": {"session_id": "abc5"}}

In [43]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response

'Hello Jim! How can I assist you today?'

# Database Integration

In [44]:
db = SQLDatabase.from_uri("sqlite:///data/Northwind_small.sqlite")

In [45]:
print(db.dialect)

sqlite


In [46]:
print(db.get_usable_table_names())

['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


In [48]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [49]:
write_query = create_sql_query_chain(llm, db)

In [50]:
response = write_query.invoke({"question": "How many customers are there"}) 
response

'SELECT COUNT("Id") AS TotalCustomers\nFROM Customer;'

In [51]:
db.run(response)

'[(91,)]'

In [52]:
execute_query = QuerySQLDataBaseTool(db=db)

/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_14043/3528740886.py:1: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


In [53]:
sql_chain = write_query | execute_query

In [54]:
sql_chain.invoke({"question": "How many employees are there"})

'[(9,)]'

In [55]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are a total of 9 employees.'

In [58]:
RunnablePassthrough.assign(query=write_query).invoke({"question": "How many employees are there"})

{'question': 'How many employees are there',
 'query': 'SELECT COUNT("Id") AS TotalEmployees FROM Employee;'}

In [59]:
RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query")).invoke({"question": "How many employees are there"})

{'question': 'How many employees are there',
 'query': 'SELECT COUNT("Id") AS TotalEmployees FROM Employee;',
 'result': 'SELECT COUNT("Id") AS TotalEmployees FROM Employee;'}

In [60]:
search = DuckDuckGoSearchRun()
search.run("When will the next solar eclipse be?")

"When's the next total solar eclipse? If seeing these amazing celestial spectacles makes you eager to see the next one, here is a short list of dates for the U.S. and Canada. For travelers, we've also highlighted three upcoming eclipses across the globe.. Future Eclipses Get Ready for These Upcoming Eclipses! Solar Eclipses ... The date listed for each eclipse is the local date where the eclipse occurs. Lunar Eclipses ... Eclipse News The next partial solar eclipse in the U.S. will occur on Aug. 12, 2026, when northeastern states see a small partial solar eclipse at lunchtime — New York 10%, Boston 16% and Bar Harbor, Maine 24%. After the 2024 total solar eclipse, astronomy lovers are eager to know when the next extraterrestrial event will be visible in the U.S. Here is the schedule for the upcoming solar eclipses. When is the next total solar eclipse? The sun, moon and Earth will align in 2026 to create a total solar eclipse, but you'll need to travel if you want to see the awe-inspir

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>